In [3]:
# COM MACD
from datetime import datetime
import MetaTrader5 as mt5
import time
import telepot
import pytz

agora = datetime.now()
print(f'Buscando dados...{agora}')
# importamos o módulo pandas para exibir os dados recebidos na forma de uma tabela
import pandas as pd
pd.set_option('display.max_columns', 500) # número de colunas
pd.set_option('display.width', 1500)      # largura máxima da tabela
pd.options.mode.chained_assignment = None  # default='warn'


if not mt5.initialize(login=1092947504, server="ClearInvestimentos-DEMO", password="Joh0516"):
    print("initialize() failed, error code =",mt5.last_error())
    quit()

symbol = ["WDOF22","WINZ21"]
#item = "WDOZ21"

for item in symbol:
    print(f'Ativo: {item}')
    timezone = pytz.timezone("Etc/UTC")
    utc_from = datetime(2021, 12, 2, tzinfo=timezone)
    rates = mt5.copy_rates_from(item, mt5.TIMEFRAME_M5, utc_from, 113)
    rates_frame = pd.DataFrame(rates)
    rates_frame['time']=pd.to_datetime(rates_frame['time'], unit='s')
    resumo = rates_frame[['time','open','close','spread']]
    #resumo

    #Criar Sinais de compra e venda
    openMME = resumo['open'].ewm(span=3).mean()      #cor: VERDE(3)
    closeMME = resumo['close'].ewm(span=30).mean()   #cor: VERMELHA(30)
    resumo.insert(loc=4,column='MMEopen',value=openMME)
    resumo.insert(loc=5,column='MMEclose',value=closeMME)
    #print(resumo.tail())

    # MACD
    resumo['EMA12'] = resumo.close.ewm(span=12).mean()
    resumo['EMA26'] = resumo.close.ewm(span=26).mean()
    resumo['MACD'] = resumo.EMA12 - resumo.EMA26
    resumo['signal'] = resumo.MACD.ewm(span=9).mean()
    resumo['histog'] = resumo['MACD'] - resumo['signal']
    #display(resumo.tail(60))
    #teste = resumo[['MACD','signal','histog']]
    #display(teste.tail(60))

    # MONITORAMENTO
    pd.options.mode.chained_assignment = None
    resumo['flag'] = ''
    resumo['sinal'] = ''

    for i in range (1, len(resumo)):
        if resumo['MMEopen'][i] > resumo['MMEclose'][i]:
            resumo['flag'][i] = 'COMPRA'
        else:
            resumo['flag'][i] = 'VENDA'

    for x in range(1,len(resumo)):
        if resumo['flag'][x] == resumo['flag'][x-1]:
            resumo['sinal'][x] = ''
        else:
            resumo['sinal'][x] = 'sinal'

    flag = resumo['flag'].iloc[-1]

    #resumo['sinal'].iloc[-1] = 'sinal'
    #resumo['flag'].iloc[-1] = 'COMPRA'

    if resumo['sinal'].iloc[-1] == 'sinal':
        bot = telepot.Bot('1852343442:AAEBBS1NjjFRIqt-XTbb3rzRxipvk8ZqI5I')
        bot.sendMessage(-351556985, f'ATENÇÃO! MUDANÇA DE STATUS: >> {item} - {flag} <<')
        print('Dados encontrados e enviados via Telegram'.upper())


    # AVISO DE SAÍDA DA OPERAÇÃO    
    compraHISTOG = resumo['histog'].iloc[-4] > 0 and resumo['histog'].iloc[-3] > 0 and resumo['histog'].iloc[-2] > 0
    compraSIGNAL = resumo['signal'].iloc[-4] > 0 and resumo['signal'].iloc[-3] > 0 and resumo['signal'].iloc[-2] > 0    
    lucroCOMPRA = compraHISTOG and resumo['histog'].iloc[-1] < 0 and compraSIGNAL and resumo['signal'].iloc[-1] > 0

    vendaHISTOG = resumo['histog'].iloc[-4] < 0 and resumo['histog'].iloc[-3] < 0 and resumo['histog'].iloc[-2] < 0
    vendaSIGNAL = resumo['signal'].iloc[-4] < 0 and resumo['signal'].iloc[-3] < 0 and resumo['signal'].iloc[-2] < 0     
    lucroVENDA = compraHISTOG and resumo['histog'].iloc[-1] > 0 and compraSIGNAL and resumo['signal'].iloc[-1] < 0


    if lucroCOMPRA == True:
        bot = telepot.Bot('1852343442:AAEBBS1NjjFRIqt-XTbb3rzRxipvk8ZqI5I')
        bot.sendMessage(-351556985, f'ATENÇÃO! LUCRO MÁXIMO NA COMPRA: >> {item} - {flag} <<')
        print('Dados encontrados e enviados via Telegram'.upper())

    if lucroVENDA == True:
        bot = telepot.Bot('1852343442:AAEBBS1NjjFRIqt-XTbb3rzRxipvk8ZqI5I')
        bot.sendMessage(-351556985, f'ATENÇÃO! LUCRO MÁXIMO NA VENDA: >> {item} - {flag} <<')
        print('Dados encontrados e enviados via Telegram'.upper())


    display(resumo.tail(3))

    time.sleep(3)

print('')
print('Script executado com sucesso!'.upper())
#time.sleep()
    

Buscando dados...2021-12-01 12:58:57.387464
Ativo: WDOF22
DADOS ENCONTRADOS E ENVIADOS VIA TELEGRAM


,time,open,close,spread,MMEopen,MMEclose,EMA12,EMA26,MACD,signal,histog,flag,sinal
110,2021-12-01 12:45:00,5660.0,5657.0,500,5657.204584,5654.325599,5655.205616,5654.414205,0.791411,0.618860,0.172551,COMPRA,
111,2021-12-01 12:50:00,5657.0,5655.0,500,5657.102292,5654.369133,5655.173983,5654.457605,0.716378,0.638364,0.078014,COMPRA,
112,2021-12-01 12:55:00,5655.0,5653.0,500,5656.051146,5654.280755,5654.839524,5654.349616,0.489908,0.608672,-0.118765,COMPRA,


Ativo: WINZ21


,time,open,close,spread,MMEopen,MMEclose,EMA12,EMA26,MACD,signal,histog,flag,sinal
110,2021-12-01 12:45:00,104015.0,104155.0,5,103991.981427,103862.528847,104050.123740,103917.370975,132.752765,167.575187,-34.822422,COMPRA,
111,2021-12-01 12:50:00,104155.0,104185.0,5,104073.490714,103883.345308,104070.873934,103937.198927,133.675007,160.795151,-27.120144,COMPRA,
112,2021-12-01 12:55:00,104185.0,104090.0,5,104129.245357,103896.684985,104073.816406,103948.519417,125.296989,153.695519,-28.398530,COMPRA,



SCRIPT EXECUTADO COM SUCESSO!


In [ ]:
resumo

In [ ]:
# RELATÓRIO DAS POSIÇÕES ABERTAS
info_posicoes = mt5.positions_get()
print(info_posicoes)
df = pd.DataFrame(list(info_posicoes), columns=info_posicoes[0]._asdict().keys())
display(df)

In [ ]:
# aviso de saída: NACIONAIS

# COM MACD
from datetime import datetime
import MetaTrader5 as mt5
import time
import telepot


agora = datetime.now()
print(f'Buscando dados...{agora}')
# importamos o módulo pandas para exibir os dados recebidos na forma de uma tabela
import pandas as pd
pd.set_option('display.max_columns', 500) # número de colunas
pd.set_option('display.width', 1500)      # largura máxima da tabela
pd.options.mode.chained_assignment = None  # default='warn'
import pytz

#if not mt5.initialize(login=50717088, server="ICMarketsSC-Demo", password="2DLJdDev"):
#if not mt5.initialize(login=54679378, server="MetaQuotes-Demo", password="hz7ulfri"):
if not mt5.initialize(login=1092947504, server="ClearInvestimentos-DEMO", password="Joh0516"):
    print("initialize() failed, error code =",mt5.last_error())
    quit()



ativos = ["WDOZ21","WDOZ21"]

for item in ativos:
    # CRIAÇÃO DOS CÁLCULOS (MÉDIAS)
    # definimos o fuso horário como UTC
    timezone = pytz.timezone("Etc/UTC")
    # criamos o objeto datetime no fuso horário UTC para que não seja aplicado o deslocamento do fuso horário local
    utc_from = datetime(2021, 11, 30, tzinfo=timezone)
    # recebemos 10 barras de EURUSD H4 a partir de 01/10/2019 no fuso horário UTC
    rates = mt5.copy_rates_from(item, mt5.TIMEFRAME_M5, utc_from, 113)
    # a partir dos dados recebidos criamos o DataFrame
    rates_frame = pd.DataFrame(rates)
    # convertemos o tempo em segundos no formato datetime
    rates_frame['time']=pd.to_datetime(rates_frame['time'], unit='s')
    resumo = rates_frame[['time','open','close','spread']]
    #resumo

    #Criar Sinais de compra e venda
    openMME = resumo['open'].ewm(span=3).mean()      #cor: VERDE(3)
    closeMME = resumo['close'].ewm(span=30).mean()   #cor: VERMELHA(30)
    resumo.insert(loc=4,column='MMEopen',value=openMME)
    resumo.insert(loc=5,column='MMEclose',value=closeMME)
    #print(resumo.tail())

    # MACD
    resumo['EMA12'] = resumo.close.ewm(span=12).mean()
    resumo['EMA26'] = resumo.close.ewm(span=26).mean()
    resumo['MACD'] = resumo.EMA12 - resumo.EMA26
    resumo['signal'] = resumo.MACD.ewm(span=9).mean()
    resumo['histog'] = resumo['MACD'] - resumo['signal']
    #display(resumo.tail(60))

    #teste = resumo[['MACD','signal','histog']]
    #display(teste.tail(60))

    # MONITORAMENTO
    pd.options.mode.chained_assignment = None
    resumo['flag'] = ''
    resumo['sinal'] = ''

    for i in range (1, len(resumo)):
        if resumo['MMEopen'][i] > resumo['MMEclose'][i]:
            resumo['flag'][i] = 'COMPRA'
        else:
            resumo['flag'][i] = 'VENDA'

    for x in range(1,len(resumo)):
        if resumo['flag'][x] == resumo['flag'][x-1]:
            resumo['sinal'][x] = ''
        else:
            resumo['sinal'][x] = 'sinal'


    # RELATÓRIO DAS POSIÇÕES
    info_posicoes = mt5.positions_get()
    if info_posicoes:
        #print(info_posicoes)
        df = pd.DataFrame(list(info_posicoes), columns=info_posicoes[0]._asdict().keys())
        display(df)
        ticket = df['ticket'].iloc[0]
        natureza = df['type'].iloc[0]

    flag = resumo['flag'].iloc[-1]

    #resumo['sinal'].iloc[-1] = 'sinal'
    #resumo['flag'].iloc[-1] = 'COMPRA'


    # AVISO DE SAÍDA DA OPERAÇÃO    
    compraHISTOG = resumo['histog'].iloc[-4] > 0 and resumo['histog'].iloc[-3] > 0 and resumo['histog'].iloc[-2] > 0
    compraSIGNAL = resumo['signal'].iloc[-4] > 0 and resumo['signal'].iloc[-3] > 0 and resumo['signal'].iloc[-2] > 0    
    lucroCOMPRA = compraHISTOG and resumo['histog'].iloc[-1] < 0 and compraSIGNAL and resumo['signal'].iloc[-1] > 0

    vendaHISTOG = resumo['histog'].iloc[-4] < 0 and resumo['histog'].iloc[-3] < 0 and resumo['histog'].iloc[-2] < 0
    vendaSIGNAL = resumo['signal'].iloc[-4] < 0 and resumo['signal'].iloc[-3] < 0 and resumo['signal'].iloc[-2] < 0     
    lucroVENDA = compraHISTOG and resumo['histog'].iloc[-1] > 0 and compraSIGNAL and resumo['signal'].iloc[-1] < 0


    if lucroCOMPRA == True:
        bot = telepot.Bot('1852343442:AAEBBS1NjjFRIqt-XTbb3rzRxipvk8ZqI5I')
        bot.sendMessage(-351556985, f'ATENÇÃO! LUCRO MÁXIMO NA COMPRA: >> {item} - {flag} <<')
        print('Dados encontrados e enviados via Telegram'.upper())

    if lucroVENDA == True:
        bot = telepot.Bot('1852343442:AAEBBS1NjjFRIqt-XTbb3rzRxipvk8ZqI5I')
        bot.sendMessage(-351556985, f'ATENÇÃO! LUCRO MÁXIMO NA VENDA: >> {item} - {flag} <<')
        print('Dados encontrados e enviados via Telegram'.upper())


    display(resumo.tail(3))

    time.sleep(2)

print('')
print('Script executado com sucesso. Fechando o programa...'.upper())
#time.sleep()